In [1]:
import matplotlib.pyplot as plt
import numpy as np
import scipy as sp
import sqlite3 as sql
import os
import pandas as pd
from datetime import datetime
from glob import glob
import sklearn as sk
import sklearn.linear_model

os.sys.path.append('../')
import PlaceCellAnalysis as pc
import utilities as u
import preprocessing as pp
import behavior as b
import BayesianDecoding as bd


%load_ext autoreload
%autoreload 2

%matplotlib inline

In [5]:
mice = ['4139190.1','4139190.3','4139212.2','4139212.4','4139219.2','4139219.3','4139224.2','4139224.3','4139224.5']
#mice = ['4139219.2']

In [3]:
df = pp.load_session_db()
df = df[df['Imaging']==1]
df = df.sort_values(['MouseName','DateTime','SessionNumber'])

    

In [4]:
def make_spline_basis(x,knots=np.arange(-.2,1,.34)):
    '''make cubic spline basis functions'''
    knotfunc = lambda k: np.power(np.multiply(x-k,(x-k)>0),3)
    spline_basis_list = [knotfunc(k) for k in knots.tolist()]
    spline_basis_list += [np.ones(x.shape[0]),x,np.power(x,2)]
    return np.array(spline_basis_list).T

In [11]:
for mouse in mice:
    df_mouse = df[df['MouseName'].str.match(mouse)]
    df_mouse = df_mouse[df_mouse['Track'].str.match('TwoTower_noTimeout') | df_mouse['Track'].str.match('TwoTower_Timeout')]
    
    for i in range(df_mouse.shape[0]):
        try:
            sess = df_mouse.iloc[i]
            VRDat,C,Cd, S, A = pp.load_scan_sess(sess)
            C=sp.stats.zscore(C,axis=0)
            trial_info, tstart_inds, teleport_inds = u.by_trial_info(VRDat)
            nm = np.unique(trial_info['morphs']).shape[0]

            pcnt = u.correct_trial_mask(trial_info['rewards'],tstart_inds,teleport_inds,C.shape[0])

            wallMorph = trial_info['morphs']+trial_info['wallJitter']
            bckgndMorph = trial_info['morphs']+trial_info['bckgndJitter']
            effMorph = trial_info['morphs']+trial_info['wallJitter']+trial_info['bckgndJitter']

            trialFR = np.zeros([tstart_inds.shape[0],C.shape[1]])
            for i,(start,stop) in enumerate(zip(tstart_inds.tolist(),teleport_inds.tolist())):
                trialFR[i,:] = C[start:stop,:].mean(axis=0)


            P,H = np.zeros([C.shape[1],]),np.zeros([C.shape[1],])
            P_corr,H_corr = np.zeros([C.shape[1],]),np.zeros([C.shape[1],])
            for c in range(0,C.shape[1]):
                if nm>2:
                    H[c], P[c] = sp.stats.kruskal(trialFR[trial_info['morphs']==0,c],trialFR[trial_info['morphs']==.25,c],
                                         trialFR[trial_info['morphs']==.5,c],trialFR[trial_info['morphs']==.75,c],
                                         trialFR[trial_info['morphs']==1,c])
                    

                    H_corr[c], P_corr[c] = sp.stats.kruskal(trialFR[(trial_info['morphs']==0) & (trial_info['rewards']>0),c],
                                         trialFR[(trial_info['morphs']==.25) & (trial_info['rewards']>0),c],
                                         trialFR[trial_info['morphs']==.5,c],
                                         trialFR[(trial_info['morphs']==.75) & (trial_info['rewards']>0),c],
                                         trialFR[(trial_info['morphs']==1.) & (trial_info['rewards']>0),c])
                else:
                    H[c], P[c] = sp.stats.kruskal(trialFR[trial_info['morphs']==0,c],
                                         trialFR[trial_info['morphs']==1,c])

                    H_corr[c], P_corr[c] = sp.stats.kruskal(trialFR[(trial_info['morphs']==0) & (trial_info['rewards']>0),c],
                                         trialFR[(trial_info['morphs']==1.) & (trial_info['rewards']>0),c])


            basis = make_spline_basis(effMorph)
            xx = np.linspace(0,1)
            xx_basis = make_spline_basis(xx)
            Yspl = np.zeros([C.shape[1],xx.shape[0]])

            for c in range(0,C.shape[1],1):
                model = sk.linear_model.LinearRegression()
                model.fit(basis, trialFR[:,c])
                y_plot = model.predict(xx_basis)
                Yspl[c,:]=y_plot

            sort = np.argsort(P)
            Ps,Hs = P[sort], H[sort]
            k = 0
            while Ps[k]<= (k+1)/C.shape[1]*.01:
                k+=1

            sort = sort[:k+1]

            path = os.path.join("G:\\My Drive\\EngramCoding\\KW\\C\\all", 
                                 "%s_%s_%d" % (sess.MouseName,sess.DateFolder,sess.SessionNumber))
            os.makedirs(path)
            np.save(os.path.join(path,"P.npy"),P)
            np.save(os.path.join(path,"H.npy"),H)
            np.save(os.path.join(path,"Yspl.npy"),Yspl)

            path = os.path.join("G:\\My Drive\\EngramCoding\\KW\\C\\corr", 
                                 "%s_%s_%d" % (sess.MouseName,sess.DateFolder,sess.SessionNumber))
            os.makedirs(path)
            np.save(os.path.join(path,"P.npy"),P_corr)
            np.save(os.path.join(path,"H.npy"),H_corr)       
        except:
            pass
        

In [ ]:
for mouse in mice:
    df_mouse = df[df['MouseName'].str.match(mouse)]
    df_mouse = df_mouse[df_mouse['Track'].str.match('TwoTower_noTimeout') | df_mouse['Track'].str.match('TwoTower_Timeout')]
    
    for i in range(df_mouse.shape[0]):
        try:
            sess = df_mouse.iloc[i]
            VRDat,C,Cd, S, A = pp.load_scan_sess(sess)
            C=sp.stats.zscore(C,axis=0)
            trial_info, tstart_inds, teleport_inds = u.by_trial_info(VRDat)
            nm = np.unique(trial_info['morphs']).shape[0]

            pcnt = u.correct_trial_mask(trial_info['rewards'],tstart_inds,teleport_inds,C.shape[0])

            wallMorph = trial_info['morphs']+trial_info['wallJitter']
            bckgndMorph = trial_info['morphs']+trial_info['bckgndJitter']
            effMorph = trial_info['morphs']+trial_info['wallJitter']+trial_info['bckgndJitter']

            trialFR = np.zeros([tstart_inds.shape[0],C.shape[1]])
            for i,(start,stop) in enumerate(zip(tstart_inds.tolist(),teleport_inds.tolist())):
                trialFR[i,:] = C[start:stop,:].mean(axis=0)


            P,H = np.zeros([C.shape[1],]),np.zeros([C.shape[1],])
            P_corr,H_corr = np.zeros([C.shape[1],]),np.zeros([C.shape[1],])
            for c in range(0,C.shape[1]):
                if nm>2:
                    H[c], P[c] = sp.stats.kruskal(trialFR[trial_info['morphs']==0,c],trialFR[trial_info['morphs']==.25,c],
                                         trialFR[trial_info['morphs']==.5,c],trialFR[trial_info['morphs']==.75,c],
                                         trialFR[trial_info['morphs']==1,c])
                    

                    H_corr[c], P_corr[c] = sp.stats.kruskal(trialFR[(trial_info['morphs']==0) & (trial_info['rewards']>0),c],
                                         trialFR[(trial_info['morphs']==.25) & (trial_info['rewards']>0),c],
                                         trialFR[trial_info['morphs']==.5,c],
                                         trialFR[(trial_info['morphs']==.75) & (trial_info['rewards']>0),c],
                                         trialFR[(trial_info['morphs']==1.) & (trial_info['rewards']>0),c])
                else:
                    H[c], P[c] = sp.stats.kruskal(trialFR[trial_info['morphs']==0,c],
                                         trialFR[trial_info['morphs']==1,c])

                    H_corr[c], P_corr[c] = sp.stats.kruskal(trialFR[(trial_info['morphs']==0) & (trial_info['rewards']>0),c],
                                         trialFR[(trial_info['morphs']==1.) & (trial_info['rewards']>0),c])


            basis = make_spline_basis(effMorph)
            xx = np.linspace(0,1)
            xx_basis = make_spline_basis(xx)
            Yspl = np.zeros([C.shape[1],xx.shape[0]])

            for c in range(0,C.shape[1],1):
                model = sk.linear_model.LinearRegression()
                model.fit(basis, trialFR[:,c])
                y_plot = model.predict(xx_basis)
                Yspl[c,:]=y_plot

            sort = np.argsort(P)
            Ps,Hs = P[sort], H[sort]
            k = 0
            while Ps[k]<= (k+1)/C.shape[1]*.01:
                k+=1

            sort = sort[:k+1]

            path = os.path.join("G:\\My Drive\\EngramCoding\\KW\\C\\all", 
                                 "%s_%s_%d" % (sess.MouseName,sess.DateFolder,sess.SessionNumber))
            os.makedirs(path)
            np.save(os.path.join(path,"P.npy"),P)
            np.save(os.path.join(path,"H.npy"),H)
            np.save(os.path.join(path,"Yspl.npy"),Yspl)

            path = os.path.join("G:\\My Drive\\EngramCoding\\KW\\C\\corr", 
                                 "%s_%s_%d" % (sess.MouseName,sess.DateFolder,sess.SessionNumber))
            os.makedirs(path)
            np.save(os.path.join(path,"P.npy"),P_corr)
            np.save(os.path.join(path,"H.npy"),H_corr)       
        except:
            pass